In [12]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [13]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [14]:
model_path = '/content/drive/MyDrive/best_lstm_complete_model.h5'
tokenizer_path = '/content/drive/MyDrive/tokenizer.pickle'
label_encoder_path = '/content/drive/MyDrive/label_encoder.pickle'

In [15]:
# -------------------------
# Text cleaning function
# -------------------------
def clean_text(text):
    """
    Lowercase, remove special characters, and strip whitespace
    """
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.strip()
    return text

In [16]:
# -------------------------
# Load model, tokenizer, and label encoder ONCE
# -------------------------

print("Loading model and preprocessing objects...")
model = tf.keras.models.load_model(model_path)
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)
with open(label_encoder_path, 'rb') as handle:
    label_encoder = pickle.load(handle)
print("Loaded successfully!")

Loading model and preprocessing objects...


Loaded successfully!


In [17]:
# -------------------------
# Prediction function
# -------------------------
def quick_category_predict(text):
    """
    Predict the category of a single text
    Returns: category, confidence
    """
    cleaned = clean_text(text)
    sequence = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(sequence, maxlen=500, padding='post', truncating='post')
    prediction = model.predict(padded, verbose=0)

    predicted_idx = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction, axis=1)[0]
    category = label_encoder.inverse_transform([predicted_idx])[0]

    return category, confidence

In [18]:
# -------------------------
# Test examples
# -------------------------
quick_texts = [
    "My new mobile phone broke",
    "india won the asia cricket cup",
    "the salary payment is today"
]

print("\n🚀 QUICK PREDICTION EXAMPLES:")
print("-" * 50)
for text in quick_texts:
    category, confidence = quick_category_predict(text)
    print(f"Text: '{text[:50]}...'")
    print(f"→ {category.upper()} ({confidence:.2%} confidence)")
    print()


🚀 QUICK PREDICTION EXAMPLES:
--------------------------------------------------
Text: 'My new mobile phone broke...'
→ TECH (89.94% confidence)

Text: 'india won the asia cricket cup...'
→ SPORT (66.74% confidence)

Text: 'the salary payment is today...'
→ BUSINESS (58.46% confidence)

